In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import joblib
import json
from tensorflow.keras.models import load_model
from src.config import (
    TEST_FILE, INPUT_SCALER_PATH, OUTPUT_SCALER_PATH,
    FEATURES_JSON_PATH, TARGET_COLUMNS, FINAL_MODEL_PATH
)
from sklearn.metrics import r2_score, mean_absolute_error

# --- Load Artifacts --- #
print("🔄 Loading model and scalers...")
model = load_model(FINAL_MODEL_PATH)
input_scaler = joblib.load(INPUT_SCALER_PATH)
output_scaler = joblib.load(OUTPUT_SCALER_PATH)
with open(FEATURES_JSON_PATH) as f:
    selected_features = json.load(f)

# --- Load and Preprocess Test Data --- #
print("📊 Loading test data...")
df_test = pd.read_csv(TEST_FILE).dropna()
X_test_raw = df_test[selected_features]
y_test_raw = df_test[TARGET_COLUMNS]

X_test_scaled = input_scaler.transform(X_test_raw)

# --- Predict --- #
print("🚀 Predicting on test data...")
y_pred_scaled = model.predict(X_test_scaled)
y_pred = output_scaler.inverse_transform(y_pred_scaled)
y_test = y_test_raw.values

# --- Visualization --- #
print("📈 Plotting predictions vs actual...")
num_outputs = len(TARGET_COLUMNS)
plt.figure(figsize=(18, 40))

for i, col in enumerate(TARGET_COLUMNS):
    plt.subplot(num_outputs, 1, i + 1)
    plt.plot(y_test[:, i], label='Actual', linewidth=2)
    plt.plot(y_pred[:, i], label='Predicted', linestyle='dashed')
    plt.title(f"{col} (R² = {r2_score(y_test[:, i], y_pred[:, i]):.3f}, MAE = {mean_absolute_error(y_test[:, i], y_pred[:, i]):.3f})")
    plt.xlabel("Sample")
    plt.ylabel(col)
    plt.legend()
    plt.tight_layout()

plt.show()

print("✅ Evaluation complete.")
